In [116]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv

In [117]:
load_dotenv()

True

In [118]:
llm_model = ChatGroq(
    model="qwen/qwen3-32b"
)

response = llm_model.invoke("What is the capital of France?")
print(response.content)

<think>
Okay, so the user is asking, "What is the capital of France?" Let me think. I remember that France is a country in Europe, and the capital is a major city there. From what I've studied before, the capital of France is Paris. But let me double-check to be sure. Sometimes countries can have different capitals due to historical reasons or something.

Wait, Paris is known for the Eiffel Tower, the Louvre Museum, and it's a very famous city. I don't think any other city in France is considered the capital. Maybe Lyon or Marseille? No, those are big cities too, but I'm pretty sure Paris is the capital. Let me recall if there's any confusion here. I think the answer is definitely Paris. Yeah, that's right. So the capital of France is Paris. I don't see any reason to doubt that. Unless there's some recent political change, but I haven't heard of anything like that. So the answer should be Paris.
</think>

The capital of France is **Paris**. It is one of the most iconic cities in the wo

In [119]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embedding_model=GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [120]:
embeded_vector = embedding_model.embed_query("What is the capital of France?")
len(embeded_vector)

768

# RAG

### Data Ingestion

In [121]:
from langchain.document_loaders import PyPDFLoader

In [122]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [123]:
import os
pdf_file_path = os.path.join(os.getcwd(), "data", "sample.pdf")

In [124]:
pdfLoader = PyPDFLoader(pdf_file_path)

In [125]:
data = pdfLoader.load()

In [126]:
len(data)

77

In [127]:
data[0].page_content

'Llama 2: Open Foundation and Fine-Tuned Chat Models\nHugo Touvron∗ Louis Martin† Kevin Stone†\nPeter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra\nPrajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen\nGuillem Cucurull David Esiobu Jude Fernandes Jeremy Fu Wenyin Fu Brian Fuller\nCynthia Gao Vedanuj Goswami Naman Goyal Anthony Hartshorn Saghar Hosseini Rui Hou\nHakan Inan Marcin Kardas Viktor Kerkez Madian Khabsa Isabel Kloumann Artem Korenev\nPunit Singh Koura Marie-Anne Lachaux Thibaut Lavril Jenya Lee Diana Liskovich\nYinghai Lu Yuning Mao Xavier Martinet Todor Mihaylov Pushkar Mishra\nIgor Molybog Yixin Nie Andrew Poulton Jeremy Reizenstein Rashi Rungta Kalyan Saladi\nAlan Schelten Ruan Silva Eric Michael Smith Ranjan Subramanian Xiaoqing Ellen Tan Binh Tang\nRoss Taylor Adina Williams Jian Xiang Kuan Puxin Xu Zheng Yan Iliyan Zarov Yuchen Zhang\nAngela Fan Melanie Kambadur Sharan Narang Aurelien Rodriguez Robert Stojnic\

In [128]:
#This are experimental values, there is no deteministic way to split the text.
text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=1000,
                chunk_overlap=150,
                length_function=len
)

In [129]:
docs = text_splitter.split_documents(data)

In [130]:
len(docs)

330

### Storing data in Vector Database(Faiss)

In [131]:
from langchain.vectorstores import FAISS

In [132]:
#InMemory Storage
vectorDB =FAISS.from_documents(
    documents=docs,
    embedding=embedding_model,
)

In [133]:
rel_doc = vectorDB.similarity_search("Training of Llama 2-Chat", k=3)

In [134]:
rel_doc[1].page_content

'Figure 4: Training ofLlama 2-Chat: This process begins with thepretraining of Llama 2using publicly\navailable online sources. Following this, we create an initial version ofLlama 2-Chatthrough the application\nof supervised fine-tuning. Subsequently, the model is iteratively refined using Reinforcement Learning\nwith Human Feedback(RLHF) methodologies, specifically through rejection sampling and Proximal Policy\nOptimization (PPO). Throughout the RLHF stage, the accumulation ofiterative reward modeling datain\nparallel with model enhancements is crucial to ensure the reward models remain within distribution.\n2 Pretraining\nTocreatethenewfamilyof Llama 2models,webeganwiththepretrainingapproachdescribedinTouvronetal.\n(2023), using an optimized auto-regressive transformer, but made several changes to improve performance.\nSpecifically, we performed more robust data cleaning, updated our data mixes, trained on 40% more total'

In [135]:
#object for retriver of relevant documents from vector database
retriever = vectorDB.as_retriever(search_type="similarity", search_kwargs={"k": 5})


In [136]:
retriever.invoke("Training of Llama 2-Chat")

[Document(id='e3f0fdb3-f309-4b9d-895d-9a9a57f51237', metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-07-20T00:30:36+00:00', 'author': '', 'keywords': '', 'moddate': '2023-07-20T00:30:36+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'c:\\2.My Data\\Course_LLMOPS Industry Ready Projects\\document_portal\\notebook\\data\\sample.pdf', 'total_pages': 77, 'page': 0, 'page_label': '1'}, page_content='Llama 2: Open Foundation and Fine-Tuned Chat Models\nHugo Touvron∗ Louis Martin† Kevin Stone†\nPeter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra\nPrajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen\nGuillem Cucurull David Esiobu Jude Fernandes Jeremy Fu Wenyin Fu Brian Fuller\nCynthia Gao Vedanuj Goswami Naman Goyal Anthony Hartshorn Saghar Hosseini Rui Hou\nH

### Question: user question  
### Context: based on the question retrieving the info from the vector database

In [146]:
prompt_template = """
        Answer the question based on the context provided below in 2 or 3 lines. 
        If the context does not contain sufficient information, respond with: 
        "I do not have enough information about this."

        Context: {context}

        Question: {question}

        Answer:
"""



In [147]:
from langchain.prompts import PromptTemplate

In [148]:
prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]   
)

In [149]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [150]:
parser=StrOutputParser()

In [151]:
#custom function to extract the page_content from the documents
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

In [152]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm_model
    | parser
)

In [153]:
response = rag_chain.invoke("tell  me about the Training of Llama 2-Chat?")
print(response)

<think>
Okay, so the user is asking about the training of Llama 2-Chat. Let me look through the provided context to find the relevant information.

First, the context mentions that Llama 2-Chat's training starts with pretraining Llama 2 using public online sources. Then they do supervised fine-tuning to create an initial version. After that, they use RLHF (Reinforcement Learning with Human Feedback) with rejection sampling and PPO. During RLHF, they accumulate reward model data in parallel with model improvements to keep the reward models in distribution. 

I need to make sure I cover all the steps: pretraining, supervised fine-tuning, then RLHF with the specific methods mentioned. Also, note the part about using synthetic constraints for training instructions like Hobbies, Language, Public Figures. They combined these to make instructions more complex. 

Wait, there's a part about the final learning rate and other hyperparameters, but maybe that's more detail than needed. The user jus